# Prize-Collecting Steiner Tree (PCSTP)

## Libs Importing

In [1]:
import sys
import os
import time
import networkx as nx

sys.path.insert(1, os.path.realpath(os.path.pardir))

In [2]:
import multiprocessing

NUM_PROCESSES = multiprocessing.cpu_count()
print("Number of cpu : ", NUM_PROCESSES)

Number of cpu :  12


In [3]:
from pcstp.instances.generator import generate_random_steiner
from pcstp.instances.reader import SteinlibReader, DatReader

from pcstp.steinertree import SteinerTreeProblem
from pcstp.solver.base import computes_steiner_cost
from pcstp.solver.aco import AntColony
from pcstp.solver.greedy_h1 import GreedyH1

from pcstp.utils.graph import preprocessing
from pcstp.utils.draw import draw_steiner_graph

## Experiments

In [4]:
SEED = 100

In [6]:
import glob

INSTANCES_PATH_PREFIX = '../data/instances/benchmark/PCSPG-CRR'
NUM_EXPERIMENTS_PER_INSTANCE = 10

all_files = glob.glob(os.path.join(INSTANCES_PATH_PREFIX, '*'))

files = all_files

networkx_history = []
solutions = {}
for filename in files:
    if filename.endswith('.xlsx'): continue
    if filename.endswith('.stp'):
        stp_reader = SteinlibReader()
    else:
        stp_reader = DatReader()

    print(f"Reading: {filename}")
    stp = stp_reader.parser(filename=filename)
    G, terminals = preprocessing(stp.graph, stp.terminals)
    stp_preprocessed = SteinerTreeProblem(graph=G, terminals=terminals)
    
    def run_experiment(experiment: int):
        start_time = time.time()
        nx_steiner_tree = nx.algorithms.approximation.steiner_tree(
            stp_preprocessed.graph,
            stp_preprocessed.terminals,
            weight='cost'
        )

        networkx_duration = time.time() - start_time
        networkx_cost = computes_steiner_cost(stp.graph, nx_steiner_tree, stp.terminals)

        history = {
            "filename": filename,
            "experiment": experiment,
            "num_nodes": stp.num_nodes,
            "num_edges": stp.num_edges,
            "num_nodes_after_preprocessing": len(stp_preprocessed.graph.nodes),
            "num_edges_after_preprocessing": len(stp_preprocessed.graph.edges),
            "terminals": stp.num_terminals,
            "steiner_cost": networkx_cost,
            "duration": networkx_duration
        }
        print('Cost: ', networkx_cost)
        return history, nx_steiner_tree.copy()
        
    experiments = range(1, NUM_EXPERIMENTS_PER_INSTANCE+1)

    with multiprocessing.Pool(processes=NUM_PROCESSES) as p:
        experiments_results = p.map(run_experiment, experiments)
    
    networkx_history.extend([result[0] for result in experiments_results])
    solutions[filename] = [result[1] for result in experiments_results]

Reading: ../data/instances/benchmark/PCSPG-CRR/C01-A.stp
Cost:  88.0
Cost:  88.0
Cost:  88.0
Cost: Cost:   88.088.0

Cost:  88.0
Cost: Cost: Cost:   88.0 88.0Cost: 88.0


 88.0
Reading: ../data/instances/benchmark/PCSPG-CRR/C05-A.stp
Cost:  1604.0
Cost:  1604.0
Cost:  Cost: 1604.0 
1604.0
Cost: Cost:   1604.0
1604.0
Cost: Cost:   1604.01604.0

Cost:  1604.0
Cost:  1604.0
Reading: ../data/instances/benchmark/PCSPG-CRR/C04-A.stp
Cost:  1110.0
Cost:  1110.0
Cost: Cost:   1110.0
1110.0
Cost: Cost:   1110.01110.0Cost: Cost: 

  1110.01110.0
Cost: 
Cost:   1110.01110.0

Reading: ../data/instances/benchmark/PCSPG-CRR/C02-A.stp
Cost: Cost:   144.0144.0

Cost:  144.0
Cost: Cost:   Cost: 144.0144.0 Cost: 

144.0
 144.0
Cost:  144.0
Cost:  144.0
Cost:  144.0
Reading: ../data/instances/benchmark/PCSPG-CRR/C03-A.stp
Cost:  Cost: 779.0 
779.0
Cost: Cost:   779.0Cost: 
 779.0
779.0
Cost:  Cost: 779.0
 779.0Cost: 
 779.0
Cost:  779.0
Cost:  779.0


In [ ]:
import pandas as pd

df_score_networkx = pd.DataFrame.from_dict(networkx_history)
df_score_networkx.to_csv(os.path.join(INSTANCES_PATH_PREFIX, 'NETWORKX.csv'))

In [ ]:
df_score_networkx.groupby('filename')[['duration', 'steiner_cost']].describe()

In [8]:
experiment = 4
G = solutions['../data/instances/benchmark/PCSPG-CRR/C03-A.stp'][experiment].graph
steiner_tree = solutions['../data/instances/benchmark/PCSPG-CRR/C03-A.stp'][experiment]

In [10]:
steiner_tree